<a href="https://colab.research.google.com/github/unt-geetha/Geetha_INFO5731_-Fall2021/blob/main/palreddy_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training.

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.

Algorithms:

(1) MultinominalNB

(2) SVM

(3) KNN

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison

(4) F-1 score

In [ ]:
# importing necessary packages and Libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
from sklearn.model_selection import cross_val_predict
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import DBSCAN
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [ ]:
# Importing File paths

path_stsa_train = 'stsa-train.txt'
path_stsa_test = 'stsa-test.txt'

# Reading train and test files directly into DataFrames
train_data_stsa = pd.read_csv(path_stsa_train, sep='\t', header=None, names=['review_label', 'review'])
test_data_stsa = pd.read_csv(path_stsa_test, sep='\t', header=None, names=['review_label', 'review'])

In [ ]:
train_data_stsa.head()

,review_label,review
0,"1 a stirring , funny and finally transporting ...",NaN
1,0 apparently reassembled from the cutting-room...,NaN
2,0 they presume their audience wo n't sit still...,NaN
3,1 this is a visually stunning rumination on lo...,NaN
4,1 jonathan parker 's bartleby should have been...,NaN


In [ ]:
test_data_stsa.head()

,review_label,review
0,"0 no movement , no yuks , not much of anything .",NaN
1,"0 a gob of drivel so sickly sweet , even the e...",NaN
2,"0 gangs of new york is an unapologetic mess , ...",NaN
3,0 we never really feel involved with the story...,NaN
4,1 this is one of polanski 's best films .,NaN


###  here, it is found that review_label and the reviews has text along with its label in same review, so need to split the train and test data

In [ ]:
# Defining a Function to read and split lines into 'review_label' and 'review'

def read_and_split(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    split_data = [line.strip().split(' ', 1) for line in lines]
    return split_data

# Reading and splitting train and test data

train_splitting_data = read_and_split(path_stsa_train)
test_splitting_data = read_and_split(path_stsa_test)

In [ ]:
train_splitting_data

[['1',
  'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films'],
 ['0',
  'apparently reassembled from the cutting-room floor of any given daytime soap .'],
 ['0',
  "they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science-fiction elements of bug-eyed monsters and futuristic women in skimpy clothes ."],
 ['1',
  'this is a visually stunning rumination on love , memory , history and the war between art and commerce .'],
 ['1',
  "jonathan parker 's bartleby should have been the be-all-end-all of the modern-office anomie films ."],
 ['1',
  'campanella gets the tone just right -- funny in the middle of sad in the middle of hopeful .'],
 ['0',
  'a fan film that for the uninitiated plays better on video with the sound turned down .'],
 ['1',
  'béart and berling are both superb , while huppert ... is magnificent .'],
 ['0',
  'a little less extreme

In [ ]:
test_splitting_data

[['0', 'no movement , no yuks , not much of anything .'],
 ['0',
  "a gob of drivel so sickly sweet , even the eager consumers of moore 's pasteurized ditties will retch it up like rancid crème brûlée ."],
 ['0',
  'gangs of new york is an unapologetic mess , whose only saving grace is that it ends by blowing just about everything up .'],
 ['0',
  'we never really feel involved with the story , as all of its ideas remain just that : abstract ideas .'],
 ['1', "this is one of polanski 's best films ."],
 ['1',
  'take care of my cat offers a refreshingly different slice of asian cinema .'],
 ['0',
  "acting , particularly by tambor , almost makes `` never again '' worthwhile , but -lrb- writer\\/director -rrb- schaeffer should follow his titular advice"],
 ['1', 'the movie exists for its soccer action and its fine acting .'],
 ['0',
  "arnold 's jump from little screen to big will leave frowns on more than a few faces ."],
 ['0',
  'if this holiday movie is supposed to be a gift , someb

In [ ]:
# Creating two DataFrames with columns with splitted data and name the columns

dataset_train = pd.DataFrame(train_splitting_data, columns=['review_label', 'review'])
dataset_test = pd.DataFrame(test_splitting_data, columns=['review_label', 'review'])

In [ ]:
# Checking head of the Training Dataset

dataset_train.head()

,review_label,review
0,1,"a stirring , funny and finally transporting re..."
1,0,apparently reassembled from the cutting-room f...
2,0,they presume their audience wo n't sit still f...
3,1,this is a visually stunning rumination on love...
4,1,jonathan parker 's bartleby should have been t...


In [ ]:
# Checking head of the Testing Dataset

dataset_test.head()

,review_label,review
0,0,"no movement , no yuks , not much of anything ."
1,0,"a gob of drivel so sickly sweet , even the eag..."
2,0,"gangs of new york is an unapologetic mess , wh..."
3,0,"we never really feel involved with the story ,..."
4,1,this is one of polanski 's best films .


In [ ]:
dataset_train.review_label.nunique()

2

In [ ]:
# Defining a funtion for Pre-Processing Text and cleaning Text

def preprocess_cleaning_text(text):

    text = re.sub(r'http\S+|www\S+', '', text)

    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = text.lower()

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))

    filtered_tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    cleaned_preprocessed_text = ' '.join(lemmatized_tokens)

    return cleaned_preprocessed_text

In [ ]:
dataset_train['cleaned_review'] = dataset_train['review'].apply(preprocess_cleaning_text)

dataset_test['cleaned_review'] = dataset_test['review'].apply(preprocess_cleaning_text)

In [ ]:
dataset_train.tail()

,review_label,review,cleaned_review
6915,1,"painful , horrifying and oppressively tragic ,...",painful horrifying oppressively tragic film mi...
6916,0,take care is nicely performed by a quintet of ...,take care nicely performed quintet actress non...
6917,0,"the script covers huge , heavy topics in a bla...",script cover huge heavy topic bland surfacey w...
6918,0,a seriously bad film with seriously warped log...,seriously bad film seriously warped logic writ...
6919,1,a deliciously nonsensical comedy about a city ...,deliciously nonsensical comedy city coming apa...


In [ ]:
dataset_test.tail()

,review_label,review,cleaned_review
1816,0,"an often-deadly boring , strange reading of a ...",oftendeadly boring strange reading classic who...
1817,0,the problem with concept films is that if the ...,problem concept film concept poor one saving m...
1818,0,"safe conduct , however ambitious and well-inte...",safe conduct however ambitious wellintentioned...
1819,0,"a film made with as little wit , interest , an...",film made little wit interest professionalism ...
1820,0,"but here 's the real damn : it is n't funny , ...",real damn nt funny either


In [ ]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review_label    6920 non-null   object
 1   review          6920 non-null   object
 2   cleaned_review  6920 non-null   object
dtypes: object(3)
memory usage: 162.3+ KB


In [ ]:
# Converting review labels from string to integers in dataset_train and dataset_test

dataset_train['review_label'] = dataset_train['review_label'].astype(int)

# Convert review labels from string to integers in dataset_test

dataset_test['review_label'] = dataset_test['review_label'].astype(int)

In [ ]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6920 entries, 0 to 6919
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   review_label    6920 non-null   int64 
 1   review          6920 non-null   object
 2   cleaned_review  6920 non-null   object
dtypes: int64(1), object(2)
memory usage: 162.3+ KB


In [ ]:
dataset_train.isna().sum()

review_label      0
review            0
cleaned_review    0
dtype: int64

In [ ]:
dataset_train = dataset_train[:200]
dataset_test = dataset_test[:200]

In [ ]:
# Splitting given data into training and testing datasets

X_train, X_val, y_train, y_val = train_test_split(dataset_train['cleaned_review'], dataset_train['review_label'], test_size=0.2, random_state=42)

In [ ]:
X_train

79     fascinating curiosity piece fascinating ten mi...
197    stick rigidly paradigm rarely permitting chara...
38     overall cletis tout winning comedy excites ima...
24     lack inspiration original bloated plot stretch...
122    rerelease ron howard apollo imax format prof a...
                             ...                        
106    vincent gallo right home french shocker playin...
14                     may genial ingenious get job done
92     playing role almost bergmanesque intensity bis...
179    cast keep pretty watchable casting mick jagger...
102    moving tale love destruction unexpected place ...
Name: cleaned_review, Length: 160, dtype: object

In [ ]:
X_val

95                                 leigh make life count
15         freedom watching stunt crude fastpaced insane
30     wellintentioned effort still burdened actor of...
158     end movie definitely convinced woman spectacular
128    movie negligible work manipulation exploitatio...
115    babyfaced renner eerily convincing bland blank...
69     ambitious project fairly inexperienced filmmak...
170                   may ask buy movie milk tv cow free
174    colorful joyous celebration life tapestry wove...
45                                             lousy one
66     uglyduckling tale hideously clumsily told feel...
182    little question serious work important directo...
165    lot effort intelligence display execution awkw...
78     jackson shamefully stroll mess smug grin inexp...
186    admirably dark first script brent hanley paxto...
177    modernday character nowhere near vivid thcentu...
56     feel lrb rrb choke leash around neck director ...
152    reminder used make movie

In [ ]:
y_train

79     0
197    0
38     1
24     0
122    1
      ..
106    1
14     1
92     1
179    1
102    1
Name: review_label, Length: 160, dtype: int64

In [ ]:
y_val

95     1
15     1
30     0
158    1
128    0
115    1
69     1
170    0
174    1
45     0
66     0
182    1
165    0
78     0
186    1
177    0
56     0
152    1
82     0
68     1
124    1
16     0
148    0
93     0
65     0
60     1
84     0
67     0
125    1
132    0
9      0
18     0
55     1
75     1
150    1
104    0
135    0
137    0
164    1
76     1
Name: review_label, dtype: int64

In [ ]:
# Initializing classifiers
classifiers = [
    MultinomialNB(),
    SVC(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    XGBClassifier()
]

from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiating TF-IDF vectorizer
tfidf = TfidfVectorizer()

# Fitting and transform on training data
X_train_tfidf = tfidf.fit_transform(X_train)

# Transforming validation data
X_val_tfidf = tfidf.transform(X_val)

# Looping through classifiers
for clf in classifiers:
    # Performing 10-fold cross-validation
    cv_scores = cross_val_score(clf, X_train_tfidf, y_train, cv=10)

    # Training the model
    clf.fit(X_train_tfidf, y_train)

    # Making predictions on validation set
    y_pred = clf.predict(X_val_tfidf)

    # Calculating evaluation metrics
    accuracy = accuracy_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    # Printing evaluation metrics and cross-validation scores
    print(f"Classifier Model: {clf.__class__.__name__}")
    print(f"Cross-validation scores: {cv_scores}")
    print(f"Accuracy: {accuracy}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F1 Score: {f1}")
    print("\n")

Classifier Model: MultinomialNB
Cross-validation scores: [0.5625 0.6875 0.5    0.5625 0.625  0.6875 0.5625 0.625  0.6875 0.6875]
Accuracy: 0.525
Recall: 1.0
Precision: 0.4864864864864865
F1 Score: 0.6545454545454547


Classifier Model: SVC
Cross-validation scores: [0.5625 0.5625 0.5625 0.5625 0.5625 0.625  0.625  0.625  0.625  0.625 ]
Accuracy: 0.5
Recall: 0.9444444444444444
Precision: 0.4722222222222222
F1 Score: 0.6296296296296297


Classifier Model: KNeighborsClassifier
Cross-validation scores: [0.75   0.4375 0.625  0.5625 0.625  0.6875 0.625  0.6875 0.5    0.5   ]
Accuracy: 0.725
Recall: 0.5555555555555556
Precision: 0.7692307692307693
F1 Score: 0.6451612903225806


Classifier Model: DecisionTreeClassifier
Cross-validation scores: [0.5625 0.375  0.625  0.5625 0.6875 0.625  0.5    0.5625 0.5    0.4375]
Accuracy: 0.625
Recall: 0.5555555555555556
Precision: 0.5882352941176471
F1 Score: 0.5714285714285715


Classifier Model: RandomForestClassifier
Cross-validation scores: [0.625  0.312

In [ ]:
# Tokenizing your text data
tokenized_text = [review.split() for review in dataset_train['cleaned_review']]

# Training Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Generating Word2Vec embeddings for entire dataset
X = np.array([np.mean([word2vec_model.wv[word] for word in review.split() if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for review in dataset_train['cleaned_review']])
y = dataset_train['review_label'].values

# Initializing and train a classifier (e.g., Logistic Regression) with cross-validation
classifier = LogisticRegression(max_iter=1000, random_state=42)

# Initializing K-fold cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Performing cross-validation and calculate evaluation metrics
accuracy_scores = cross_val_score(classifier, X, y, cv=kfold, scoring='accuracy')
precision_scores = cross_val_score(classifier, X, y, cv=kfold, scoring='precision')
recall_scores = cross_val_score(classifier, X, y, cv=kfold, scoring='recall')
f1_scores = cross_val_score(classifier, X, y, cv=kfold, scoring='f1')

# Printing cross-validation scores
print("Model: Word2Vec")
print("Accuracy:", np.mean(accuracy_scores))
print("Precision:", np.mean(precision_scores))
print("Recall:", np.mean(recall_scores))
print("F1-score:", np.mean(f1_scores))

Model: Word2Vec
Accuracy: 0.565
Precision: 0.565
Recall: 1.0
F1-score: 0.7217741935483871


In [ ]:
def bert_cross_validation(X, y):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

    def tokenize(text):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        return inputs

    X_tokenized = [tokenize(text) for text in X]

    def predict(inputs):
        outputs = model(**inputs)
        return outputs.logits.detach().numpy()

    X_vec = [predict(inputs) for inputs in X_tokenized]
    X_vec = np.array(X_vec).squeeze()

    clf = RandomForestClassifier()

    # Performing cross-validation
    y_pred = cross_val_predict(clf, X_vec, y, cv=10)

    # Calculating evaluation metrics
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    return accuracy, precision, recall, f1

# Performing cross-validation for BERT models
accuracy, precision, recall, f1 = bert_cross_validation(X_train, y_train)

print("BERT Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model Evaluation Metrics:
Accuracy: 0.51875
Precision: 0.5849056603773585
Recall: 0.6526315789473685
F1-score: 0.6169154228855722


(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [ ]:
#Write your code here.

amazon_reviews = pd.read_csv('Amazon_Unlocked_Mobile.csv')

In [ ]:
amazon_reviews = amazon_reviews[:200]

In [ ]:
amazon_reviews.tail()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
195,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,2,My Granny couldn't use the phone because she w...,1.0
196,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,2,"Not good sound, not easily used by senior, end...",0.0
197,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,5,"really surprised by this phone, with fm radio ...",0.0
198,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,5,Nice big numbers everywhere! Simple too!,0.0
199,[XMAS DEAL] Jethro [SC118] Simple Unlocked Qua...,Jethro,59.99,1,Ordered phone but it would not work with AT&T....,0.0


In [ ]:
amazon_reviews.isna().sum()

Product Name    0
Brand Name      8
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [ ]:

amazon_reviews['clean_text'] = amazon_reviews['Reviews'].apply(preprocess_cleaning_text)

In [ ]:
# Creating TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(amazon_reviews['clean_text'])

In [ ]:
def tfidf_kmeans_clustering(data, max_features=1000, n_clusters=5):
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=90)
    kmeans.fit(tfidf_matrix)

    return kmeans.labels_

In [ ]:
def tfidf_dbscan_clustering(data, eps=0.5, min_samples=5):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(tfidf_matrix)

    return dbscan.labels_

In [ ]:
def tfidf_hierarchical_clustering(data, n_clusters=5):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    hierarchical = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    hierarchical_clusters = hierarchical.fit_predict(tfidf_matrix.toarray())

    return hierarchical_clusters

In [ ]:
def word2vec_kmeans_clustering(data, vector_size=100, window=5, min_count=1, n_clusters=5):
    tokenized_corpus = [text.split() for text in data]
    word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=vector_size, window=window, min_count=min_count, sg=1)

    X_vec = word_averaging_list(word2vec_model.wv, tokenized_corpus)

    kmeans_word2vec = KMeans(n_clusters=n_clusters, init='k-means++', random_state=90)
    kmeans_word2vec.fit(X_vec)

    return kmeans_word2vec.labels_

# Generating embeddings for each review
def word_averaging(wv, words):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.key_to_index:
            mean.append(wv[word])
    if not mean:
        return np.zeros(wv.vector_size,)
    mean = np.array(mean).mean(axis=0)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list])

In [ ]:
def bert_kmeans_clustering(data, tokenizer, model, n_clusters=5):
    def bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        mean_embeddings = torch.mean(last_hidden_states, dim=1)
        return mean_embeddings.detach().numpy()

    X_bert = np.vstack([bert_embedding(text) for text in data])

    kmeans_bert = KMeans(n_clusters=n_clusters, init='k-means++', random_state=90)
    kmeans_bert.fit(X_bert)

    return kmeans_bert.labels_

In [ ]:
tfidf_kmeans_labels = tfidf_kmeans_clustering(amazon_reviews['clean_text'], max_features=1000, n_clusters=5)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
tfidf_kmeans_labels

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
       0, 4, 0, 1, 0, 4, 1, 0, 0, 0, 0, 4, 4, 4, 4, 0, 3, 3, 2, 2, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 3, 0, 0, 2, 3, 2, 0, 1, 1, 3, 2,
       2, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 2, 0, 2, 4, 1, 0, 3, 0, 3, 0,
       0, 0, 4, 4, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 3, 0,
       0, 0, 1, 3, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 3, 1, 0, 0, 0,
       0, 0, 0, 3, 4, 0, 0, 3, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

In [ ]:
tfidf_dbscan_labels = tfidf_dbscan_clustering(amazon_reviews['clean_text'], eps=0.5, min_samples=5)

In [ ]:
tfidf_dbscan_labels

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  0, -1,  0, -1, -1, -1, -1,  0,  0, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,
        0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
tfidf_hierarchical_labels = tfidf_hierarchical_clustering(amazon_reviews['clean_text'], n_clusters=5)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [ ]:
tfidf_hierarchical_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 4, 0,
       0, 4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 3, 0, 1, 0, 2, 3,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4,
       0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 3, 0, 3, 0, 1, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [ ]:
word2vec_kmeans_labels = word2vec_kmeans_clustering(amazon_reviews['clean_text'], vector_size=100, window=5, min_count=1, n_clusters=5)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
word2vec_kmeans_labels

array([4, 4, 2, 3, 3, 4, 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 3, 3, 3, 4, 3, 0,
       4, 3, 3, 4, 0, 1, 3, 3, 0, 0, 3, 1, 3, 3, 0, 4, 0, 4, 2, 2, 1, 0,
       3, 3, 4, 3, 1, 2, 0, 3, 4, 2, 3, 4, 0, 4, 2, 4, 2, 1, 4, 0, 0, 2,
       2, 4, 4, 4, 0, 2, 0, 2, 0, 4, 4, 4, 4, 0, 4, 0, 1, 4, 4, 0, 4, 3,
       4, 4, 0, 4, 1, 0, 0, 4, 0, 4, 4, 3, 2, 4, 2, 4, 4, 0, 4, 0, 4, 4,
       3, 0, 0, 3, 3, 3, 2, 0, 2, 4, 0, 4, 2, 3, 3, 4, 0, 0, 4, 0, 3, 3,
       0, 3, 3, 0, 2, 2, 2, 1, 0, 3, 4, 4, 4, 4, 4, 1, 3, 4, 0, 3, 0, 4,
       3, 4, 3, 4, 0, 0, 4, 3, 0, 4, 0, 4, 3, 3, 0, 0, 3, 0, 1, 4, 4, 4,
       4, 4, 2, 0, 1, 0, 4, 3, 3, 1, 2, 4, 4, 3, 0, 2, 0, 3, 3, 1, 0, 3,
       4, 4], dtype=int32)

In [ ]:
# Loading pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

bert_kmeans_labels = bert_kmeans_clustering(amazon_reviews['clean_text'], tokenizer, model, n_clusters=5)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
bert_kmeans_labels

array([1, 1, 0, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 4, 4, 1, 4, 0, 1, 1, 4, 1, 4, 1, 4, 0, 3, 3, 4, 0,
       1, 0, 4, 4, 0, 0, 4, 0, 1, 2, 1, 0, 1, 1, 3, 1, 3, 0, 0, 4, 4, 3,
       3, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 4, 0, 0, 4, 1, 1, 1, 3, 1, 3, 1, 0, 1, 4, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 4, 3, 1, 1, 4, 4, 1, 1, 1, 0, 4, 1, 1, 0, 1,
       0, 1, 0, 4, 2, 0, 2, 0, 1, 1, 1, 1, 0, 1, 1, 4, 1, 1, 1, 1, 4, 0,
       4, 1, 1, 4, 4, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 4, 1, 0, 4, 0, 1, 4,
       1, 1, 4, 1, 0, 1, 4, 1, 4, 0, 3, 4, 1, 1, 0, 4, 4, 1, 1, 1, 1, 1,
       4, 1], dtype=int32)

In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

In [ ]:
#You can write you answer here. (No code needed)

'''
K-means clustering is a popular method for unsupervised learning that partitions a dataset into a predefined number of clusters. DBSCAN clustering is a density-based clustering algorithm that identifies clusters based on the density of points in the dataset. Hierarchical clustering is a bottom-up approach that builds a hierarchy of clusters based on the similarity between points. Word2Vec is a word embedding algorithm that represents words as vectors in a high-dimensional space. BERT is a pre-trained language model that can be used for a variety of natural language processing tasks, including text clustering.
'''


'\nK-means clustering is a popular method for unsupervised learning that partitions a dataset into a predefined number of clusters. DBSCAN clustering is a density-based clustering algorithm that identifies clusters based on the density of points in the dataset. Hierarchical clustering is a bottom-up approach that builds a hierarchy of clusters based on the similarity between points. Word2Vec is a word embedding algorithm that represents words as vectors in a high-dimensional space. BERT is a pre-trained language model that can be used for a variety of natural language processing tasks, including text clustering.\n'